# Cube with Planar Varrying Cross Section

## Cross Section Choice

For this model, I have chose to use a total cross section defined as

$$\Sigma_t(\boldsymbol{r}) = x + y + z + 1.$$

This cross section will be valid over a $2cm \times 2cm \times 2cm$ cube, with a lower corner at $(0,0,0)$ and and upper corner at (2,2,2). It will be a single energy group, non-multiplying system, with the following scatter and capture cross sections:

$$ \frac{\Sigma_s(\boldsymbol{r})}{\Sigma_t(\boldsymbol{r})} = 0.7 \qquad \frac{\Sigma_\gamma(\boldsymbol{r})}{\Sigma_t(\boldsymbol{r})} = 0.3$$

The cube will be divided into 32 sections along each axis, creating 32768 smaller cubes, in which the cross sections shall be constant. As such, a separate cross section definition must be created for each smaller cube. The following code does just that for openMC, using the cross section values at the center of a given cube.

In [1]:
import os
import numpy as np
import openmc

%matplotlib inline

In [2]:
# Define the interval for the single group
E_groups = openmc.mgxs.EnergyGroups(np.logspace(-5,7,3))

# Number of bins on each axis
Nx = 5
Ny = 5
Nz = 5

# Loop to generate XS data for each cube
dx = 2.0/Nx
dy = 2.0/Ny
dz = 2.0/Nz
xs_data_file = openmc.MGXSLibrary(E_groups)
for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            x = i*dx + (dx/2.0)
            y = j*dy + (dy/2.0)
            z = k*dz + (dz/2.0)
            XSname = str(i) + '.' + str(j) + '.' + str(k)
            XS = openmc.XSdata(XSname, E_groups)
            XS.order = 0
            Et = [(x + y + z + 1.0),(x + y + z + 1.0)]
            Es = [[[0.35*Et[0], 0.35*Et[1]],
                   [0.35*Et[0], 0.35*Et[1]]]]
            Es = np.array(Es)
            Es = np.rollaxis(Es, 0, 3)
            Ec = [0.3*Et[0], 0.3*Et[1]]
            XS.set_total(Et, temperature=294.)
            XS.set_absorption(Ec, temperature=294.)
            XS.set_scatter_matrix(Es, temperature=294.)
            xs_data_file.add_xsdata(XS)
            
# Write XS data to .h5 file
xs_data_file.export_to_hdf5('cube_plane_xs.h5')

# Now must define materials
materials = {}
for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            XSname = str(i) + '.' + str(j) + '.' + str(k)
            materials[XSname] = openmc.Material(name = XSname)
            materials[XSname].set_density('macro', 1.)
            materials[XSname].add_macroscopic(XSname)
            
materials_file = openmc.Materials(materials.values())
materials_file.cross_sections = 'cube_plane_xs.h5'
materials_file.export_to_xml()

## Creation of Surfaces and Cells

In [3]:
# Create all x-surfaces
XSurfaces = {}
Nxsurfs = Nx + 1
for i in range(Nxsurfs):
    x = i*dx;
    if((i == 0) or (i == Nxsurfs - 1)):
        XSurfaces[i] = openmc.XPlane(x0=x, boundary_type='vacuum')
    else:
        XSurfaces[i] = openmc.XPlane(x0=x)
        
# Create all y-surfaces
YSurfaces = {}
Nysurfs = Ny + 1
for j in range(Nysurfs):
    y = j*dy;
    if((j == 0) or (j == Nysurfs - 1)):
        YSurfaces[j] = openmc.YPlane(y0=y, boundary_type='vacuum')
    else:
        YSurfaces[j] = openmc.YPlane(y0=y)
        
# Create all z-surfaces
ZSurfaces = {}
Nzsurfs = Nz + 1
for k in range(Nzsurfs):
    z = k*dz;
    if((k == 0) or (k == Nysurfs - 1)):
        ZSurfaces[k] = openmc.ZPlane(z0=z, boundary_type='vacuum')
    else:
        ZSurfaces[k] = openmc.ZPlane(z0=z)
        
# Create all cells
Cells = []
for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            XSname = str(i) + '.' + str(j) + '.' + str(k)
            cell = openmc.Cell()
            cell.fill = materials[XSname]
            cell.region = +XSurfaces[i] & -XSurfaces[i+1] & +YSurfaces[j] & -YSurfaces[j+1] & +ZSurfaces[k] & -ZSurfaces[k+1]
            Cells.append(cell)

# Main universe
Box = openmc.Universe(cells=Cells)

# Make geometry file
geometry = openmc.Geometry(Box)
geometry.export_to_xml()

## Leakage Tallies

In [4]:
# Filter mesh for outter surfaces
mesh = openmc.RegularMesh(mesh_id=1)
mesh.dimension = [1,1,1]
mesh.lower_left = [0.,0.,0.]
mesh.width = [2.,2.,2.]
meshsurface_filter = openmc.MeshSurfaceFilter(mesh)

leakage = openmc.Tally(name='leakage')
leakage.filters = [meshsurface_filter]
leakage.scores = ['current']

# Tallies file
tallies = openmc.Tallies()
tallies.append(leakage)
tallies.export_to_xml()

## Settings

In [5]:
settings = openmc.Settings()

settings.run_mode = 'fixed source'
settings.energy_mode = 'multi-group'
settings.particles = 1000000
settings.batches = 1
settings.verbosity = 4

# Source definition: Point at (1,1,1) isotropic
source = openmc.Source()
source.space = openmc.stats.Point(xyz=(1.,1.,1.))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([10.0e6],[1.])
settings.source = source

settings.export_to_xml()